# Restraint as cause of death

This looks at cases where retraint was an official cause of death. This relates to the story [Held down and breathless: When police restraint turns fatal](http://www.mystatesman.com/news/crime--law/held-down-and-breathless-when-police-restraint-turns-fatal/RZwAkn7h0JpDRIWxHEyBtJ/).

The ID for [Retraint](http://data.statesman.com/admin/custody_deaths/cause/2/change/) in the Causes table is 2.

In [1]:
import agate

In [2]:
# sets data types on fields agate got wrong
specified_data_types = {
    'tracked_cause': agate.Text(),
    'offense': agate.Text(),
    'case_study': agate.Text(),
    'official_discipline': agate.Text()
}

# this pulls the deaths file that is exported in the File processing notebook
deaths = agate.Table.from_csv('../exports/deaths_latest.csv', column_types=specified_data_types)

print(deaths)

| column               | data_type |
| -------------------- | --------- |
| id                   | Number    |
| ag_report_url        | Text      |
| first_name           | Text      |
| middle_name          | Text      |
| last_name            | Text      |
| suffix               | Text      |
| slug                 | Text      |
| race                 | Text      |
| gender               | Text      |
| date_of_birth        | Date      |
| date_of_death        | Date      |
| age                  | Number    |
| agency               | Number    |
| restrained           | Boolean   |
| tazed                | Boolean   |
| times_tazed          | Number    |
| pepper_sprayed       | Boolean   |
| official_discipline  | Text      |
| grand_jury_result    | Text      |
| mental_health_issues | Boolean   |
| manner_of_death      | Text      |
| drug_intoxication    | Boolean   |
| cause_of_death       | Text      |
| tracked_cause        | Text      |
| offense              | Text      |
|

In [3]:
# functon to check if tracked_cause includes 2 - Restraint
value_checked = ['2']

# This is the evaluation function, used to compare offenses in a row
# to the list of minor offenses
def list_check(row):
    if row is None:
        return False
    myList = row.split(',')
    for item in myList:
        if item in value_checked:
            return True

# this filters our death_offenses table based on the 
# value and field we are searching
deaths_restraint = deaths.where(lambda row: list_check(row['tracked_cause']))

print('\nNumber of cases with restraint as cause:\n{}\n\nHere is a list:\n'.format(len(deaths_restraint)))

# prints the deaths with just minor offenses
deaths_restraint.select(['first_name',
                  'middle_name',
                  'last_name',
                  'date_of_death',
                  'tracked_cause']).order_by('date_of_death').print_table(max_rows=None)



Number of cases with restraint as cause:
52

Here is a list:

| first_name  | middle_name | last_name        | date_of_death | tracked_cause |
| ----------- | ----------- | ---------------- | ------------- | ------------- |
| Curtis      | Lee         | Lewis            |    2005-01-10 | 1,2           |
| Joel        | Don         | Casey            |    2005-02-18 | 2             |
| Edgar       | Alejandro   | Vera             |    2005-08-19 | 2             |
| Anthony     | George      | Brown            |    2006-05-07 | 2             |
| Curtis      | Michael     | Harwell          |    2006-09-14 | 2             |
| Herman      |             | Barnes           |    2006-10-06 | 2             |
| Rosendo     |             | Gaytan           |    2006-11-05 | 2             |
| Paul        |             | Rodriguez        |    2007-03-26 | 1,2           |
| Marcus      | Nygel       | Elliott          |    2007-07-01 | 2             |
| Donald      | Wayne       | Jacobs          

## Restraint deaths by race

In [4]:
# pivot restraint deaths by race
deaths_restraint_race = deaths_restraint.pivot('race_name')

deaths_restraint_race.to_csv('../exports/deaths_restraint_race.csv')

print('Out of all deaths ({}), {} have restraint as a cause of death.\n'.format(
        len(deaths),
        len(deaths_restraint)
    ))

print('Breaking these {} by race looks like this:\n'.format(
    len(deaths_restraint)
    ))

deaths_restraint_race.print_table()


print('\nCompared to the race of all {} cases:\n'.format(
        len(deaths)
    ))

deaths.pivot('race_name').print_table()

Out of all deaths (289), 52 have restraint as a cause of death.

Breaking these 52 by race looks like this:

| race_name       | Count |
| --------------- | ----- |
| White           |    17 |
| Black           |    21 |
| Hispanic/Latino |    14 |

Compared to the race of all 289 cases:

| race_name       | Count |
| --------------- | ----- |
| White           |    93 |
| Black           |   101 |
| Hispanic/Latino |    91 |
| Asian           |     4 |


## Restraint deaths by year


In [5]:
# pivot restraint deaths by year
# add a year column based on year of date_of_death
deaths_restraint_yearadd = deaths_restraint.compute([
    ('Year', agate.Formula(agate.Text(), lambda r: r['date_of_death'].year))
])

# pivot based on year
deaths_restraint_year = deaths_restraint_yearadd.pivot('Year').order_by('Year')

# export the data
deaths_restraint_year.to_csv('../exports/deaths_restraint_year.csv')

print('Looking at the {} restraint deaths by year:\n'.format(
        len(deaths_restraint)
    ))

deaths_restraint_year.print_table()

print('\nThen those {} as a simple bar chart:\n'.format(
        len(deaths_restraint)
    ))

# # print the pivot as bars
deaths_restraint_year.print_bars('Year', 'Count', width=60)

Looking at the 52 restraint deaths by year:

| Year | Count |
| ---- | ----- |
| 2005 |     3 |
| 2006 |     4 |
| 2007 |     3 |
| 2008 |     3 |
| 2009 |     3 |
| 2010 |     3 |
| 2011 |     6 |
| 2012 |     7 |
| 2013 |     5 |
| 2014 |     3 |
| 2015 |     9 |
| 2016 |     3 |

Then those 52 as a simple bar chart:

Year Count
2005     3 ▓░░░░░░░░░░░░░░░░                                
2006     4 ▓░░░░░░░░░░░░░░░░░░░░░                           
2007     3 ▓░░░░░░░░░░░░░░░░                                
2008     3 ▓░░░░░░░░░░░░░░░░                                
2009     3 ▓░░░░░░░░░░░░░░░░                                
2010     3 ▓░░░░░░░░░░░░░░░░                                
2011     6 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                
2012     7 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░           
2013     5 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░                     
2014     3 ▓░░░░░░░░░░░░░░░░                                
2015     9 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░

## Manner of death for those under restraint

In [6]:
deaths_restraint.pivot('manner_name').print_table()

| manner_name  | Count |
| ------------ | ----- |
| Undetermined |     8 |
| Homicide     |    42 |
| Accident     |     1 |
| Natural      |     1 |
